<a href="https://colab.research.google.com/github/shilosh/ContinuousLST/blob/master/ContLST_GeoTIFF_to_NETCDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert tiff to NetCDF using xarray:

connecting gdrive into the google colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Test gdrive connection

In [ ]:
!ls '/content/gdrive/My Drive/MODIS/2000'

In [ ]:
!pip install xarray
!pip install affine
!pip install Rasterio 
!pip install netcdf4
!!pip install geopandas 


Import libraries

In [ ]:
from affine import Affine
import glob
import pandas as pd
import xarray as xr
#import xgeo # Needs to be imported to use geo extension
import geopandas as gpd
import os

function to read the date from file names:

In [ ]:


def time_index_from_filenames(filenames):
    '''helper function to create a pandas DatetimeIndex
       Filename example: LST_2002_12_30.tif'''
    #return pd.DatetimeIndex([pd.Timestamp(year=f[4:8], month=f[9:11], day=f[12:14], hour=12) for f in filenames])
    names = [os.path.basename(x) for x in filenames]
    return pd.DatetimeIndex([pd.Timestamp(f[4:8]+f[9:11]+f[12:14]) for f in names])
    

Iterate through all LSTcont folders in Google Drive and create 1 NetCDF from each folder (1 file for entire year)

In [ ]:
for year in range (2017, 2018):
  filenames = sorted(glob.glob('/content/gdrive/My Drive/MODIS/' + str(year) + '/*.tif'))
  print(filenames)
  time = xr.Variable('time', time_index_from_filenames(filenames))
  chunks = {'x': 409, 'y': 603}
  da = xr.concat([xr.open_rasterio(f, chunks=chunks) for f in filenames], dim=time)
  da.to_netcdf(path='/content/gdrive/My Drive/MODIS/' + str(year) + '.nc', format='NETCDF4', mode='w', encoding={'__xarray_dataarray_variable__': {'dtype': 'float32', 'scale_factor': 0.02}}) 
  print(year)

['/content/gdrive/My Drive/MODIS/2017/LST_2017_01_01.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_02.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_03.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_04.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_05.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_06.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_07.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_08.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_09.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_10.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_11.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_12.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_13.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_14.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_15.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_16.tif', '/content/gdrive/My Drive/MODIS/2017/LST_2017_01_17.tif', '/content/gdr

# Convert tiff to NetCDF using [gdal_translate](https://gdal.org/programs/gdal_translate.html#gdal-translate):

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import gdal

In [ ]:
filename = '/content/gdrive/My Drive/MODIS/LSTconst_validation_full.tif'
da = gdal.Open(filename)
out_ds = gdal.Translate('/content/gdrive/My Drive/MODIS/LSTconst_validation.nc', da, options= "bandList=[1,2,3] format='NC4'")
print(da.RasterCount)
print(out_ds)